# Defacing pre-registration - Statistical analysis on image quality metrics (IQMs) in python

## Load IQMs from MRIQC output

In [ ]:
## LOAD IQMs

randomization_path = "../randomization/"
data_path = "/data/derivatives/mriqc/RoyalCarpetPlot/DefacingPilotData/shuffled"
manualQCpathCP = "/data/code/DefacingProject/ManualQC/Celine"
manualQCpathOE = "/data/code/DefacingProject/ManualQC/Oscar"

with open(os.path.join(randomization_path,"DefacingPilotData_blind_dict.json")) as json_file:
    blind_dict = json.load(json_file)
with open(os.path.join(randomization_path,"DefacingPilotData_pos_dict.json")) as json_file:
    pos_dict = json.load(json_file)
 
anonym_dict = {'{}'.format(i) : i+1 for i in range(0,9)}
print(anonym_dict)
def get_key(my_dict,val):
    for key, value in my_dict.items():
         if val == value:
             return key
        
df_api = pd.read_pickle('./MRIQC_API_T1w/T1w_siemens_prisma')
df_api = df_api.reindex(sorted(df_api.columns), axis=1)
api = df_api.to_numpy()

manual_original = np.zeros((10,2))
manual_defaced = np.zeros((10,2))  
iqms_original = np.zeros((10,61))
iqms_defaced = np.zeros((10,61))
for s in range(0,20):
    with open(os.path.join(data_path, 'sub-{}'.format(s), "ses-V1", "anat","sub-{}_ses-V1_run-1_T1w.json".format(s))) as json_file:
        iqms = json.load(json_file)
        iqms_keys = list(iqms.keys())
        #Drop non IQMs field
        iqms_keys.remove('bids_meta')
        iqms_keys.remove('provenance')
        iqms_keys.remove('size_x')
        iqms_keys.remove('size_y')
        iqms_keys.remove('size_z')
        iqms_keys.remove('spacing_x')
        iqms_keys.remove('spacing_y')
        iqms_keys.remove('spacing_z')
        iqms_keys.remove('summary_bg_p05')
        
    assert len(df_api.columns) == len(iqms_keys), 'There is not the same nbr of iqms in the API based and in this dataset'

    sub = get_key(blind_dict,s)
    pos = int(get_key(pos_dict,sub[0:8]))

    if "non_deface" in sub:
        # Manual quality rating
        manual_original[pos,0] = scoreCP['rating']
        manual_original[pos,1] = scoreOE['rating']
        # Automatic quality metrics
        for i,key in enumerate(iqms_keys):
            iqms_original[pos,i] = iqms[key]
    elif "pydeface" in sub:
        # Manual quality rating
        manual_defaced[pos,0] = scoreCP['rating']
        manual_defaced[pos,1] = scoreOE['rating']
        # Automatic quality metrics
        for i,key in enumerate(iqms_keys):
            iqms_defaced[pos,i] = iqms[key]
    else:
        raise ValueError("{} is an invalid name".format(sub)) 

## Run MANOVA

In [ ]:
# Build dataframe
i_o = np.hstack((iqms_original,np.zeros((10,1))))
i_d = np.hstack((iqms_defaced,np.ones((10,1))))
print(i_o.shape)
print(i_d.shape)
i_merge = np.vstack((i_o,i_d))
#Verify shape matches expectation
print(i_merge.shape)

iqms_df = pd.DataFrame(i_merge, columns = iqms_keys + ['Defaced'])
print(iqms_keys)

#Construct formula
dep_var = '' 
for key in iqms_keys:
    dep_var += '{} +'.format(key)
#Remove the last ' +'
dep_var = dep_var[:-2]

#Run MANOVA
maov = MANOVA.from_formula('{} ~ Defaced'.format(dep_var), data=iqms_df)

print(maov.mv_test())